In [1]:
# Network Algorithms Homework Project


In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# Obtain the data
airports = pd.read_csv('airports.csv')
routes = pd.read_csv('routes.csv')

In [4]:
def filter_na(df, column):
    na_rows = df[column].str.len() < 3
    return df[~na_rows]

# Filter the N/A values in the data
filtered_airports = filter_na(airports, "IATA")
filtered_routes = filter_na(routes, 'Source airport ID')
filtered_routes = filter_na(filtered_routes, 'Destination airport ID')

In [5]:
def get_countries_of(region):
    if (region == 'Europe'):
        import requests
        response = requests.get("https://restcountries.com/v3.1/region/europe")
        l = response.json()
        europe = []
        for i in range(len(l)):
            europe.append(l[i]['name']['common'])
        return europe
    # future support for other regions

def filter_to_europe(df, attr):
    europe = get_countries_of('Europe')
    return df[df[attr].isin(europe)]

# Choose only one region (europe in our example)
filtered_european_airports = filter_to_europe(filtered_airports, 'Country')

In [6]:
# Filter routes according to filtered airports
iatas = filtered_european_airports['IATA']
filtered_european_routes = filtered_routes[filtered_routes['Source airport'].isin(iatas) & filtered_routes['Destination airport'].isin(iatas)]

In [ ]:

# returns all the information about an airport by its IATA
def airport_info(data_airports, iata):
    return data_airports.loc[data_airports['IATA'] == iata]

# Raw DataFrame to be cleaned  
data_airports = pd.read_csv('airports.csv')
data_routes = pd.read_csv('routes.csv')

# to display all the columns
# pd.set_option('display.max_columns', None)

# for checking
print(data_airports.head())
print(data_routes.head())

# for checking
print(airport_info(data_airports,'GYD'))
print(airport_info(data_airports,'HEL'))
print(airport_info(data_airports,'NRT'))

# array dist is an array of all the distances between sources and destinations that should be created and filled by Rustam
# once it's filled a new data frame with the Source, Destination and Distance values will be created
dist = []
data = [data_routes['Source airport'],data_routes['Destination airport'],dist]

# new DataFrame only with the Source, Destination and Distance values
df = pd.DataFrame(data, columns = ['Source', 'Dest', 'Dist'])